# 21M.387 Fundamentals of Music Processing
## Glossary


## 2 Kick / Snare Classifier

$N$ : number of samples

$F_s$ : sampling rate

$L$ : window size

$H$ : hop size

$M$ : number hops, $M = \lfloor {N - L \over H} \rfloor + 1$

$F_f$ : feature rate




## 3 Fourier Transform

DFT: $X(k) = \sum_{n=0}^{N-1}x(n)e^{-j 2 \pi kn / N } \text{, for } k \in [0:N-1]$

- $N$ : number of samples in DFT (which is also the number of samples in the original signal)  
- Frequency at $k$: $f_k = k {F_s \over N}$

Sinusoid: $x(t) = A \cos(\omega t + \phi)$

- $\omega$:  _angular velocity_.
- $\omega = 2 \pi f$, where $f$ is frequency in _Hertz_
- $\phi \in [- \pi, \pi)$: _phase_.
- $A$: amplitude


Discrete Sinusoid: $x(n) = A \cos(2 \pi {k \over N} n + \phi)$  

Dot product: $\langle x, s \rangle = \sum_{n=0}^{N-1}x(n) \cdot s(n)$

FFT : Optimized version of FFT taking advantage of symmetrical nature of DFT using divide and conquer

- DFT is $O(N^2)$
- FFT is $O(N \log N)$ if $N$ is a power of 2.

STFT: $\mathcal{X}(n,k) = \sum_{l=0}^{N-1}x(l+nH)w(l)e^{-j 2 \pi kl / N }$

- $N$: length of DFT
- $H$: hop size
- $n$: time step (a "Hop")
- $w(l)$: a window function (like a _Hann_ window)
- $f_k = k F_s / N$: the frequency at $k$
- $T_n = n H / F_s$: time time at $n$

Spectrogram: $\vert \mathcal{X}(n,k) \lvert^2$

Gamma compression: $\Gamma_\gamma(v) = \log(1+ \gamma \cdot v)$

- $v$: value to compress
- $\gamma$: compression factor

Zero padding: can be used to increase resolution of DFT. You're not getting more information but a finer resolution of the same information


## 4 Chromagrams / DTW


_Chromagram_: captures the pitch-content and harmonic content of a signal.
- insensitive to octave
- insensitive to timbre
- they are also a model more closely related to how the ear hears pitch.

_Pitchogram_: also known as the _Log Frequency Spectrogram_: buckets frequencies together based on the "midi" pitch.

$\mathbf{P} = \mathbf{C}_{fp} \cdot \mathbf{X}$
- $\mathbf{P} \text{ the pitch-o-gram, is a } P \times L$ matrix  
- $\mathbf{C}_{fp} \text{ the conversion matrix, is a } P \times K$ matrix  
- $\mathbf{X} \text{ the spectrogram, is a } K \times L$ matrix  
- $L$ is the length of the spectrogram (number of "hops")  
- $K$ is the number of frequency bins. $K = 1 + N/2$.  
- $P$ is the number of midi pitches. $P = 128$.



_Chromagram computation_: Reduce the pitch-o-gram by collapsing all pitches to their respective pitch class.

$\mathbf{C} = \mathbf{C}_{fc} \cdot \mathbf{X} $
- $\mathbf{C}$ is the chromagram, a $12 \times L$ matrix
- $\mathbf{C}_{fc}$ is the conversion matrix from spectrogram to chromagram, a $12 \times K$ matrix

_Dynamic Time Warping_ attempts to find a mapping between two similar pieces of audio that are placed differently in time. 

Steps:
1. Compute chromagram of both signals
2. Creat "cost matrix", $\mathbf{C}$, where every time index in chromagram 1 is compared to every time index in chromagram 2.
3. The comparison we use is the "cosine distance"
$$\mathbf{C}(n,m) = 1 - {\langle x_n, y_m \rangle \over {\Vert x_n \Vert \cdot \Vert y_m \Vert} }$$
4. Define path constraints
    - It must start at $p_1 = (1,1)$ and end at $p_L = (N,M)$.
    - It can't go backwards.
    - It can only have certain step sizes. We will use these:
      - $\Sigma = \{(1,0), (0,1), (1,1)\}$
5. Find shortest path using dynamic programming 
$$\mathbf{D}(n,1) = \sum_{k=1}^n \mathbf{C}(k,1) \text{ for } n \in[1:N] $$
$$\mathbf{D}(1,m) = \sum_{k=1}^m \mathbf{C}(1,k) \text{ for } m \in[1:M] $$
$$\mathbf{D}(n,m) = \mathbf{C}(n,m) + \mathrm{min}
\begin{cases}
\mathbf{D}(n-1,m) \\
\mathbf{D}(n,m-1) \\
\mathbf{D}(n-1,m-1) \\
\end{cases}$$
6. $\mathbf{D}(n,m)$ gives us the lowest cumulative cost from $(1,1)$  to $(N,M)$
7. Use backtracking to find path by keeping track of which step was taken during each subproblem


## 5 Beat Tracking

<img src="5_beat/images/beat_tracking_overview.png">

$\Delta^s(n)$ : spectral novelty function

Steps to compute:
1. Create the spectrogram of the signal
2. Apply Log Compression
3. Frame-wise differentiation and half-wave rectification
4. Accumulate all rows
5. Improve curve by subtracting local average

### Fourier Tempogram

Take the STFT of $\Delta^s(n)$, the feature signal! 

__Must be careful about units!__

$F_s$ : sampling frequency of audio

$H$: Hop size for spectral novelty curve calculation

$F_f = F_s/H$ : sampling frequency of spectral novelty curve 

$H_{tg}$: Hop size for tempogram calculation

$F_{tg} = F_f/H_{tg}$ : sampling frequency of tempogram

### Autocorrelation Tempogram

An alternative to taking the STFT of spectral novelty function, we can take autocorrelation of each window.

$$ R_{xx}(l) = \sum_{n=0}^{L-1} x(n) x(n-l) $$

This tempogram is indexed by $(n,l)$:

- $n$ is "time" - horizontal
- $l$ is "lag" - vertical.


The time-axis is the same as the STFT tempogram: $t = n / F_{tg}$

The frequency at each bin is the inverse: $ f_l = {F_f \over l }$

### Predominant Local Pulse

1. Find the peak frequency of the tempogram for each window.
1. Synthesize synthetic sinusoid matching frequency of window.
1. Combine all these synhtetic sinusoids into one
1. Pick peaks!


### Beat Detection by Dynamic Programming

The goal is to create a _beat sequence_. Let's call that $B$:  
$B = (b_1, b_2, b_3, ... b_L)$

Each $b_l$ is a _beat location_ - a timestamp in the Novelty Curve $\Delta(n)$.  
There are $L$ beats in the beat sequence.

$\hat \delta$ is the tempo estimate in units of samples. To compute it:  


$\mathbf{S}(B) = [\text{beats align with onsets}] + \lambda [\text{beat deltas} \simeq \hat \delta]$
$$\mathbf{S}(B) = \sum_{l=1}^L \Delta(b_l) + \lambda \sum_{l=2}^L P_{\hat \delta}(b_l - b_{l-1})$$

$\lambda$ weights the importance of consistency of the beat versus evidence from the song on where beats actually lie  

$P_{\hat \delta}(\delta)$ is the penalty function
$$P_{\hat \delta}(\delta) = -(\log_2(\delta / \hat \delta))^2$$ 

We want to solve:
$$B^* = \text{argmax}\lbrace \mathbf{S}(B) \mid B \in \mathcal{B}^N\rbrace$$

Use Dynamic Programming!

$$\mathbf{D}(0) = 0$$

$$\mathbf{D}(n) = \max \lbrace \mathbf{S}(B) \mid B \in \mathcal{B}^N_n \rbrace$$

$$
\mathbf{D}(n) = \Delta(n) + \max
\begin{cases}
0,  \\
\max_{m \in [1:n-1]} \lbrace \mathbf{D}(m) + \lambda P_{\hat \delta}(n - m)\rbrace \\
\end{cases}
$$
